In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
class ConstructDataset:
    def __init__(self, data_dir, scenario='neutral'):
        # Directory containing core data files
        self.data_dir = data_dir
        
        # Demand scenario (either 'neutral' or 'low')
        self.scenario = scenario
        
        # Filenames
        # ---------
        # NTNDP database spreasheets
        self.ntndp_filename = '2016 Planning Studies - Additional Modelling Data and Assumptions summary.xlsm'
        
        # ACIL Allen spreadsheets
        self.acil_filename = 'Fuel_and_Technology_Cost_Review_Data_ACIL_Allen.xlsx'
        
        
        # Mappings
        # --------
        # Map between zones and regions
        self.df_zones_map = pd.read_csv(os.path.join(data_dir, 'maps', 'zones.csv'), index_col=0)

        # Map between wind bubbles, zones and regions
        self.df_bubble_map = pd.read_csv(os.path.join(data_dir, 'maps', 'bubbles.csv'), index_col=0)

        # Map between thermal unit types, fuel categories, and unit categories
        self.df_thermal_map = pd.read_csv(os.path.join(data_dir, 'maps', 'thermal_unit_types.csv'), index_col=0)
        
        # Map between existing gas and coal generators and fuel cost IDs
        self.df_fuel_cost_map = pd.read_csv(os.path.join(data_dir, 'maps', 'existing_fuel_cost_map.csv'), index_col=0)
        
        # Map between candidate coal generator IDs and fuel cost IDs
        self.df_ntndp_coal_cost_map = pd.read_csv(os.path.join(data_dir, 'maps', 'candidate_coal_cost_map.csv'), index_col=0)
        
        # Map between existing generators and IDs for FOM costs in NTNDP database
        self.df_fom_cost_map = pd.read_csv(os.path.join(data_dir, 'maps', 'existing_fom_cost_map.csv'), index_col=0)
        
        
        # NTNDP Data
        # ----------
        # Load fixed operating and maintenance cost data
        self.df_ntndp_fom = self._load_ntndp_fom()
        
        # Variable operating and maintenance cost data
        self.df_ntndp_vom = self._load_ntndp_vom()
                
        # Heat rates
        self.df_ntndp_heat_rates = self._load_ntndp_heat_rates()
        
        # Emissions
        self.df_ntndp_emissions = self._load_ntndp_emissions_rates()
        
        # Build costs
        self.df_ntndp_build_cost = self._load_ntndp_build_costs_neutral()
        
        # Coal cost
        self.df_ntndp_coal_cost = self._load_ntndp_coal_cost_neutral()
        
        # Gas cost
        self.df_ntndp_gas_cost = self._load_ntndp_gas_cost_neutral()
        
        
        # ACIL Allen Data
        # ---------------
        self.df_acil_fuel_cost = self._load_acil_existing_fuel_cost()
        
        
        # NEM Dataset
        # -----------
        # All existing generators from NEM dataset
        self.df_g = pd.read_csv(os.path.join(data_dir, 'files', 'generators.csv'), index_col='DUID')
        
                
    def _load_ntndp_fom(self):
        """Load fixed operating and maintenance cost spreadsheet from NTNDP database"""
        
        # Fixed operating and maintenance costs
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='FOM', skiprows=2, header=None)
              .rename(columns={0: 'NTNDP_ID', 1: 'FOM'}).set_index('NTNDP_ID'))
        
        return df
    
    def _load_ntndp_vom(self):
        """Load variable operating and maintenance (VOM) cost spreadsheet from NTNDP database"""
        
        # Variable operating and maintenance costs
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='VOM', skiprows=2, header=None)
              .rename(columns={0: 'NTNDP_ID', 1: 'VOM'}).set_index('NTNDP_ID'))
        
        return df
    
    def _load_ntndp_heat_rates(self):
        """Load heat rate spreadsheet from NTNDP database"""
        
        # Heat rates
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Heat Rates', skiprows=1))
        
        # Rename columns and set index
        df = (df.rename(columns={'Generators': 'NTNDP_ID', 'Heat Rate (GJ/MWh)': 'HEAT_RATE',
                                'Date From': 'DATE_FROM', 'Date To': 'DATE_TO',
                                'Category': 'CATEGORY'})
              .set_index('NTNDP_ID'))
        
        return df
    
    def _load_ntndp_emissions_rates(self):
        """Load emissions rate data from NTNDP database"""

        # Emissions rates
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Emissions Rate', skiprows=1))
        
        # Rename columns and set index
        df = (df.rename(columns={'Generator': 'NTNDP_ID', 'Comb Co2 (kg/MWh)': 'EMISSIONS',
                                'Date From': 'DATE_FROM',
                                'Fugi Co2 (kg/MWh)': 'FUGITIVE_EMISSIONS'})
              .set_index('NTNDP_ID'))
        
        return df
    
    def _load_ntndp_build_costs_neutral(self):
        """Load candidate unit build costs (assuming neutral demand scenario)"""
    
        # Build costs - neutral demand scenario
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Build Cost', skiprows=3, nrows=137))
        
        # Rename columns and set index
        df = df.rename(columns={'$/kW': 'NTNDP_ID'}).set_index('NTNDP_ID')
        
        return df
    
    def _load_ntndp_build_costs_low(self):
        """Load candidate unit build costs (assuming low demand scenario)"""
    
        # Build costs - low demand scenario
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Build Cost', skiprows=144, nrows=137))
        
        # Rename columns and set index
        df = df.rename(columns={'$/kW': 'NTNDP_ID'}).set_index('NTNDP_ID')
        
        return df
    
    def _load_ntndp_coal_cost_neutral(self):
        """Coal cost information (neutral demand scenario)"""
        
        # Coal costs - neutral demand scenario
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Coal Cost', skiprows=2, nrows=22))
        
        # Rename columns and set index
        df = df.rename(columns={'Fuel Cost ($/GJ)': 'FUEL_COST_ID'}).set_index('FUEL_COST_ID')
        
        return df
    
    def _load_ntndp_coal_cost_low(self):
        """Coal cost information (low demand scenario)"""
        
        # Coal costs - low demand scenario
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Coal Cost', skiprows=28, nrows=22))
        
        # Rename columns and set index
        df = df.rename(columns={'Fuel Cost ($/GJ)': 'FUEL_COST_ID'}).set_index('FUEL_COST_ID')
        
        return df
    
    def _load_ntndp_gas_cost_neutral(self):
        """Gas cost information (neutral demand scenario)"""
        
        # Gas costs - neutral demand scenario
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Gas Cost', skiprows=2, nrows=63))
        
        # Rename columns and set index
        df = df.rename(columns={'Fuel Cost ($/GJ)': 'FUEL_COST_ID'}).set_index('FUEL_COST_ID')
        
        return df

    def _load_ntndp_gas_cost_low(self):
        """Gas cost information (low demand scenario)"""
        
        # Gas costs - low demand scenario
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.ntndp_filename),
                            sheet_name='Gas Cost', skiprows=69, nrows=63))
        
        # Rename columns and set index
        df = df.rename(columns={'Fuel Cost ($/GJ)': 'FUEL_COST_ID'}).set_index('FUEL_COST_ID')
        
        return df
    
    def _load_acil_existing_fuel_cost(self):
        """Existing fuel costs"""
        
        # Load 
        df = (pd.read_excel(os.path.join(self.data_dir, 'files', self.acil_filename),
                            sheet_name='Existing Fuel Costs', skiprows=2, nrows=154))
        
        # Rename columns and set index
        df = df.rename(columns={'Profile': 'FUEL_COST_ID'}).set_index('FUEL_COST_ID')
        
        return df
    
    def get_existing_duids(self, fuel_type):
        """
        Get existing generator DUIDs by fuel type
        
        Parameters
        ----------
        fuel_type : str
            Type of fuel. Options: 'ALL', 'COAL', 'GAS', 'LIQUID', 'HYDRO', 'WIND'
        
        Returns
        -------
        duids : list
            List of existing generators with corresponding fuel type
        """
        
        # Filter for existing coal units
        mask_coal = self.df_g['FUEL_TYPE'].isin(['Brown coal', 'Black coal'])
        
        # Filter for exsiting gas units
        mask_gas = self.df_g['FUEL_TYPE'].isin(['Natural Gas (Pipeline)', 'Coal seam methane'])
        
        # Filter for exising liquid (e.g. diesel) units
        mask_liquid = self.df_g['FUEL_TYPE'].isin(['Kerosene - non aviation', 'Diesel oil'])
        
        # Filter for exsiting wind units
        mask_wind = self.df_g['FUEL_TYPE'].isin(['Wind'])
        
        # Filter for existing hydro units
        mask_hydro = self.df_g['FUEL_TYPE'].isin(['Hydro'])
        
        if fuel_type == 'ALL':
            duids = self.df_g.index
        
        elif fuel_type == 'COAL':
            duids = self.df_g.loc[mask_coal, :].index
        
        elif fuel_type == 'GAS':
            duids = self.df_g.loc[mask_gas, :].index
        
        elif fuel_type == 'LIQUID':
            duids = self.df_g.loc[mask_liquid, :].index
        
        elif fuel_type == 'HYDRO':
            duids = self.df_g.loc[mask_hydro, :].index
        
        elif fuel_type == 'WIND':
            duids = self.df_g.loc[mask_wind, :].index
        
        else:
            raise(Exception(f"Unexpected fuel type: '{fuel_type}' encountered"))
        
        return duids   
    
    def get_all_candidate_units(self):
        """Extract information for all candidate units"""

        # All candidate unit IDs from FOM spreadsheet (use this as the basis
        # for candidate unit IDs)
        all_candidate_units = self.df_ntndp_fom[329:].index

        # Candidate units and their respective locations (could be a wind bubble for wind generators, or zone ID)
        df_candidate = pd.DataFrame([(c, c.split(' ')[0]) for c in all_candidate_units], columns=['NTNDP_UNIT_ID', 'LOCATION'])

        # Merge zone IDs - mapping wind bubbles to NEM zones
        df_candidate = pd.merge(df_candidate, self.df_bubble_map[['ZONE']], how='left', left_on=['LOCATION'], right_index=True)

        # Fill missing zones (LOCATION and ZONE will be the same for these generators)
        df_candidate['ZONE'] = df_candidate.apply(lambda x: x['LOCATION'] if pd.isnull(x['ZONE']) else x['ZONE'], axis=1)

        # Check that zone assigned to all candidate units
        assert not df_candidate['ZONE'].isna().any()

        def _get_candidate_information(row):
            """Extract unit information from unit names"""

            if 'WIND' in row['NTNDP_UNIT_ID']:
                gen_type, fuel_cat = 'WIND', 'WIND'

            elif 'Solar' in row['NTNDP_UNIT_ID']:
                gen_type, fuel_cat = 'SOLAR', 'SOLAR'

            elif 'OCGT' in row['NTNDP_UNIT_ID']:
                gen_type, fuel_cat = 'OCGT', 'GAS'

            elif 'CCGT' in row['NTNDP_UNIT_ID']:
                gen_type, fuel_cat = 'CCGT', 'GAS'

            elif 'Coal' in row['NTNDP_UNIT_ID']:
                gen_type, fuel_cat = 'COAL', 'COAL'

            elif 'Biomass' in row['NTNDP_UNIT_ID']:
                gen_type, fuel_cat = 'BIOMASS', 'BIOMASS'

            else:
                fuel_type, fuel_cat = np.nan, np.nan

            return pd.Series(data={'GEN_TYPE': gen_type, 'FUEL_CAT': fuel_cat})

        # Get candidate generator and fuel type
        df_candidate[['GEN_TYPE', 'FUEL_CAT']] = df_candidate.apply(_get_candidate_information, axis=1)

        return df_candidate
    
    def get_candidate_wind_units(self):
        """Get candidate wind generators"""
        
        # All candidate wind generators
        df = self.get_all_candidate_units().copy()
        mask_wind = df['GEN_TYPE'] == 'WIND'

        # Only retain one wind bubble per zone
        df_wind = df.loc[mask_wind, :].drop_duplicates(subset=['ZONE'], keep='first')

        # Candidate wind units
        df_wind['UNIT_ID'] = df_wind.apply(lambda x: x['ZONE']+'-'+x['GEN_TYPE'], axis=1)
        
        # Set index and rename columns
        df_wind = df_wind.set_index('UNIT_ID').rename(columns={'LOCATION': 'WIND_BUBBLE'})
        
        return df_wind
    
    def get_candidate_solar_units(self):
        """Get candidate solar generators"""
        
        # All solar units
        df = self.get_all_candidate_units().copy()
        mask_solar = df['GEN_TYPE'] == 'SOLAR'

        # Remove generators with 40pc in name (duplicates)
        mask_40pc = df['NTNDP_UNIT_ID'].str.contains('40pc')

        # All solar units
        df_solar = df.loc[mask_solar & ~mask_40pc, :].copy()

        # Construct unit ID from zone name and solar unit type
        df_solar['UNIT_ID'] = df_solar['NTNDP_UNIT_ID'].apply(lambda x: x.upper().replace(' ', '-'))

        # Set index and drop redundant columns
        df_solar = df_solar.set_index('UNIT_ID').drop('LOCATION', axis=1)
        
        return df_solar
    
    def get_candidate_coal_units(self):
        """Get candidate coal units"""

        # Old candidate units
        df = self.get_all_candidate_units().copy()

        # Candidate coal generators
        mask_coal = df['FUEL_CAT'] == 'COAL'
        df_coal = df.loc[mask_coal, :].copy()

        # Construct unit ID
        df_coal['UNIT_ID'] = df_coal['NTNDP_UNIT_ID'].apply(lambda x: x.upper().replace(' ', '-'))

        # Set index and drop columns
        df_coal = df_coal.set_index('UNIT_ID').drop('LOCATION', axis=1)

        return df_coal
    
    def get_candidate_gas_units(self):
        """Get candidate gas units"""

        # Old candidate units
        df = self.get_all_candidate_units().copy()

        # Candidate coal generators
        mask_gas = df['FUEL_CAT'] == 'GAS'
        df_gas = df.loc[mask_gas, :].copy()

        # Construct unit ID
        df_gas['UNIT_ID'] = df_gas['NTNDP_UNIT_ID'].apply(lambda x: x.upper().replace(' ', '-'))

        # Set index and drop columns
        df_gas = df_gas.set_index('UNIT_ID').drop('LOCATION', axis=1)

        return df_gas
    
    def get_all_units(self):
        """Get all existing and candidate unit IDs"""
        
        # All unit IDs
        all_units = (self.df_g.index
                     .union(self.get_candidate_coal_units().index)
                     .union(self.get_candidate_gas_units().index)
                     .union(self.get_candidate_solar_units().index)
                     .union(self.get_candidate_wind_units().index))
        
        return all_units
    
    def get_candidate_coal_fuel_cost_profiles(self):
        """Get candidate coal generator fuel cost profiles"""

        # All candidate coal generators
        df = self.get_candidate_coal_units().copy()

        # Add fuel cost ID for each candidate generator
        df['FUEL_COST_ID'] = df.apply(lambda x: self.df_ntndp_coal_cost_map.loc[x.name, 'NTNDP_FUEL_COST_ID'], axis=1)

        # Coal cost profiles
        df_coal_cost = self.df_ntndp_coal_cost.copy()

        # Update columns
        new_cols = {i: int(i.split('-')[0]) for i in df_coal_cost}
        df_coal_cost = df_coal_cost.rename(columns=new_cols)

        # Candidate coal generator fuel costs
        df_candidate_coal_cost = (pd.merge(df[['FUEL_COST_ID']],df_coal_cost,
                                           how='left', left_on='FUEL_COST_ID', right_index=True)
                                  .drop('FUEL_COST_ID', axis=1))
        
        # Check for missing values
        assert not df_candidate_coal_cost.isna().any().any(), 'Missing coal fuel cost profile values'

        return df_candidate_coal_cost
    
    def get_candidate_gas_fuel_cost_profiles(self):
        """Get candidate generator fuel cost profiles"""

        # All candidate gas generators
        df = self.get_candidate_gas_units().copy()

        def _get_gas_fuel_cost_id(row):
            """Construct gas fuel cost ID from unit ID"""

            # NTNDP fuel cost ID
            fuel_cost_id = row.name.replace('-', ' ').replace('CCS', '(CCS)')

            return fuel_cost_id

        df['FUEL_COST_ID'] = df.apply(_get_gas_fuel_cost_id, axis=1)

        # Must manually correct these fuel cost ID assignments
        manual_map = [
            ('NCEN-CCGT', 'NCEN CCGT (CCS)'),
            ('NNS-CCGT', 'NNS CCGT (CCS)'),
            ('CQ-CCGT', 'CQ CCGT (CCS)'),
            ('NQ-CCGT', 'NQ CCGT (CCS)'),
            ('SEQ-CCGT', 'SEQ CCGT (CCS)'),
            ('SWQ-CCGT', 'SWQ CCGT (CCS)'),
            ('SESA-CCGT', 'SESA CCGT (CCS)'),
            ('LV-CCGT', 'LV CCGT (CCS)'),
            ('TAS-CCGT', 'TAS CCGT (CCS)')
        ]

        # Update fuel cost IDs for selected gas generators
        for unit_id, fuel_cost_id in manual_map:
            # Add fuel cost ID
            df.loc[unit_id, 'FUEL_COST_ID'] = fuel_cost_id

        # Gas cost
        df_gas_cost = self.df_ntndp_gas_cost.copy()

        # Rename columns
        new_cols = {i: int(i.split('-')[0]) for i in df_gas_cost.columns}
        df_gas_cost = df_gas_cost.rename(columns=new_cols)

        # Candidate gas cost
        df_candidate_gas_cost = (pd.merge(df[['FUEL_COST_ID']], df_gas_cost, how='left', 
                                          left_on=['FUEL_COST_ID'], right_index=True)
                                 .drop('FUEL_COST_ID', axis=1))

        # Check for missing values
        assert not df_candidate_gas_cost.isna().any().any(), 'Missing gas fuel cost profile values'

        return df_candidate_gas_cost
               
    def get_candidate_gas_heat_rates(self):
        """Get heat rates for candidate gas generators"""
        
        # All candidate gas generators
        df = self.get_candidate_gas_units().copy()
        
        # Heat rates for all generators
        df_heat_rates = self.df_ntndp_heat_rates.copy()

        # Remove duplicates
        df_heat_rates = df_heat_rates[~df_heat_rates.index.duplicated(keep='first')]

        df_candidate_gas_heat_rates = (pd.merge(df, df_heat_rates[['HEAT_RATE']], 
                                                how='left', left_on=['NTNDP_UNIT_ID'], 
                                                right_index=True)[['HEAT_RATE']]
                                      )
        # Check for missing values
        assert not df_candidate_gas_heat_rates['HEAT_RATE'].isna().any(), 'Missing candidate gas generator heat rates'
        
        return df_candidate_gas_heat_rates
       
    def get_candidate_coal_heat_rates(self):
        """Get heat rates for candidate gas generators"""
        
        # All candidate coal generators
        df = self.get_candidate_coal_units().copy()
        
        # Heat rates for all generators
        df_heat_rates = self.df_ntndp_heat_rates.copy()

        # Remove duplicates
        df_heat_rates = df_heat_rates[~df_heat_rates.index.duplicated(keep='first')]

        df_candidate_coal_heat_rates = (pd.merge(df, df_heat_rates[['HEAT_RATE']], 
                                                how='left', left_on=['NTNDP_UNIT_ID'], 
                                                right_index=True)[['HEAT_RATE']]
                                      )
        
        # Check for missing values
        assert not df_candidate_coal_heat_rates['HEAT_RATE'].isna().any(), 'Missing candidate coal generator heat rates'
        
        return df_candidate_coal_heat_rates

    def get_candidate_solar_heat_rates(self):
        """Get heat rates for candidate solar generators (assume=0)"""
        
        # Get all candidate solar units
        df = self.get_candidate_solar_units().copy()

        # Set emission=0 for all solar units
        df['HEAT_RATE'] = 0

        # Only retain 'emissions' column
        df_o = df[['HEAT_RATE']]
        
        return df_o
    
    def get_candidate_wind_heat_rates(self):
        """Get heat rates for candidate wind generators (assume=0)"""
        
        # Get all candidate solar units
        df = self.get_candidate_wind_units().copy()

        # Set emission=0 for all wind units
        df['HEAT_RATE'] = 0

        # Only retain 'emissions' column
        df_o = df[['HEAT_RATE']]
        
        return df_o
    
    def get_candidate_wind_vom_cost(self):
        """Get candidate wind generator variable operating and maintenance cost"""
        
        # All candidate wind generators
        df = self.get_candidate_wind_units().copy()

        # Variable operating and maintenance cost
        df_vom = pd.merge(df, self.df_ntndp_vom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['VOM']]

        # Check for missing values
        assert not df_vom.isna().any().any(), 'Missing wind VOM values'
        
        return df_vom
    
    def get_candidate_solar_vom_cost(self):
        """Get candidate solar generator variable operating and maintenance cost"""
        
        # All candidate solar generators
        df = self.get_candidate_solar_units().copy()

        # Variable operating and maintenance cost
        df_vom = pd.merge(df, self.df_ntndp_vom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['VOM']]

        # Check for missing values
        assert not df_vom.isna().any().any(), 'Missing solar VOM values'
        
        return df_vom
    
    def get_candidate_coal_vom_cost(self):
        """Get candidate coal generator variable operating and maintenance cost"""
        
        # All candidate coal generators
        df = self.get_candidate_coal_units().copy()

        # Variable operating and maintenance cost
        df_vom = pd.merge(df, self.df_ntndp_vom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['VOM']]

        # Check for missing values
        assert not df_vom.isna().any().any(), 'Missing coal VOM values'
        
        return df_vom
    
    def get_candidate_gas_vom_cost(self):
        """Get candidate gas generator variable operating and maintenance cost"""
        
        # All candidate gas generators
        df = self.get_candidate_gas_units().copy()

        # Variable operating and maintenance cost
        df_vom = pd.merge(df, self.df_ntndp_vom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['VOM']]
    
        # Check for missing values
        assert not df_vom.isna().any().any(), 'Missing gas VOM values'
        
        return df_vom
    
    def get_candidate_wind_fom_cost(self):
        """Get candidate wind generator fixed operating and maintenance cost"""
        
        # All candidate wind generators
        df = self.get_candidate_wind_units().copy()

        # Variable operating and maintenance cost
        df_fom = pd.merge(df, self.df_ntndp_fom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['FOM']]

        # Check for missing values
        assert not df_fom.isna().any().any(), 'Missing wind FOM values'
        
        return df_fom
    
    def get_candidate_solar_fom_cost(self):
        """Get candidate solar generator fixed operating and maintenance cost"""
        
        # All candidate solar generators
        df = self.get_candidate_solar_units().copy()

        # Variable operating and maintenance cost
        df_fom = pd.merge(df, self.df_ntndp_fom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['FOM']]

        # Check for missing values
        assert not df_fom.isna().any().any(), 'Missing solar FOM values'
        
        return df_fom
    
    def get_candidate_coal_fom_cost(self):
        """Get candidate coal generator fixed operating and maintenance cost"""
        
        # All candidate coal generators
        df = self.get_candidate_coal_units().copy()

        # Variable operating and maintenance cost
        df_fom = pd.merge(df, self.df_ntndp_fom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['FOM']]

        # Check for missing values
        assert not df_fom.isna().any().any(), 'Missing coal FOM values'
        
        return df_fom
    
    def get_candidate_gas_fom_cost(self):
        """Get candidate gas generator fixed operating and maintenance cost"""
        
        # All candidate gas generators
        df = self.get_candidate_gas_units().copy()

        # Variable operating and maintenance cost
        df_fom = pd.merge(df, self.df_ntndp_fom, how='left', left_on='NTNDP_UNIT_ID', right_index=True)[['FOM']]
    
        # Check for missing values
        assert not df_fom.isna().any().any(), 'Missing gas FOM values'
        
        return df_fom
    
    def get_candidate_coal_emissions_rates(self):
        """Get emissions rates for candidate coal generators"""

        # NTNDP emissions
        df_emissions = self.df_ntndp_emissions.copy()

        # Remove duplicates
        df_emissions = df_emissions.loc[~df_emissions.index.duplicated(keep='first'), :]

        # Divide kgCO2/MWh by 1000 to get tCO2/MWh
        df_emissions[['EMISSIONS', 'FUGITIVE_EMISSIONS']] = df_emissions[['EMISSIONS', 'FUGITIVE_EMISSIONS']].div(1000)

        # Candidate coal units
        df = self.get_candidate_coal_units()

        # Join candidate coal emissions rates
        df_coal_emissions = pd.merge(df[['NTNDP_UNIT_ID']], df_emissions, how='left', left_on=['NTNDP_UNIT_ID'], right_index=True)[['EMISSIONS']]

        assert not df_coal_emissions.isna().any().any(), 'Missing candidate coal emissions rates'

        return df_coal_emissions
    
    def get_candidate_gas_emissions_rates(self):
        """Get emissions rates for candidate gas generators"""

        # NTNDP emissions
        df_emissions = self.df_ntndp_emissions.copy()

        # Remove duplicates
        df_emissions = df_emissions.loc[~df_emissions.index.duplicated(keep='first'), :]

        # Divide kgCO2/MWh by 1000 to get tCO2/MWh
        df_emissions[['EMISSIONS', 'FUGITIVE_EMISSIONS']] = df_emissions[['EMISSIONS', 'FUGITIVE_EMISSIONS']].div(1000)

        # Candidate coal units
        df = self.get_candidate_gas_units()

        # Join candidate coal emissions rates
        df_gas_emissions = pd.merge(df[['NTNDP_UNIT_ID']], df_emissions, how='left', left_on=['NTNDP_UNIT_ID'], right_index=True)[['EMISSIONS']]

        assert not df_gas_emissions.isna().any().any(), 'Missing candidate gas emissions rates'

        return df_gas_emissions
    
    def get_candidate_solar_emissions_rates(self):
        """Get emissions rates for candidate solar generators (assume=0)"""
        
        # Get all candidate solar units
        df = self.get_candidate_solar_units().copy()

        # Set emission=0 for all solar units
        df['EMISSIONS'] = 0

        # Only retain 'emissions' column
        df_o = df[['EMISSIONS']]
        
        return df_o
    
    def get_candidate_wind_emissions_rates(self):
        """Get emissions rates for candidate wind generators (assume=0)"""
        
        # Get all candidate wind units
        df = self.get_candidate_wind_units().copy()

        # Set emission=0 for all solar units
        df['EMISSIONS'] = 0

        # Only retain 'emissions' column
        df_o = df[['EMISSIONS']]
        
        return df_o

    def get_existing_coal_fuel_cost_profiles(self):
        """Get fuel cost profiles for existing coal generators"""

        # Get DUIDs for existing coal generators
        existing_coal_duids = self.get_existing_duids('COAL')

        # All existing coal generators
        df = self.df_g.reindex(existing_coal_duids).copy()

        # Join fuel cost profile IDs
        df = df.join(self.df_fuel_cost_map[['FUEL_COST_ID']], how='left')

        # Check every existing coal generator has a corresponding fuel cost profile ID
        assert not df['FUEL_COST_ID'].isna().any(), 'Existing coal generator missing fuel cost ID'

        # Existing coal generators
        df_existing_coal = pd.merge(df[['FUEL_COST_ID']], self.df_ntndp_coal_cost, how='left', 
                                    left_on=['FUEL_COST_ID'], right_index=True).drop('FUEL_COST_ID', axis=1)

        # Rename columns
        new_columns = {i: int(i.split('-')[0]) for i in df_existing_coal.columns}
        df_existing_coal = df_existing_coal.rename(columns=new_columns)

        # Check for missing values
        assert not df_existing_coal.isna().any().any(), 'Missing fuel cost values for existing coal generators'

        return df_existing_coal
    
    def get_existing_gas_and_liquid_fuel_cost_profiles(self):
        """Get fuel cost profiles for existing gas and liquid fuel generators"""

        # Get DUIDs for existing gas and liquid fuel generators
        existing_gas_duids = self.get_existing_duids('GAS')
        existing_liquid_fuel_duids = self.get_existing_duids('LIQUID')

        # All existing gas and liquid fuel generators
        df = self.df_g.reindex(existing_gas_duids | existing_liquid_fuel_duids).copy()

        # Join fuel cost profile IDs
        df = df.join(self.df_fuel_cost_map[['FUEL_COST_ID']], how='left')

        # Check every existing gas and liquid fuel generator has a corresponding fuel cost profile ID
        assert not df['FUEL_COST_ID'].isna().any(), 'Existing gas generator missing fuel cost ID'

        # Existing gas generators
        df_existing = pd.merge(df[['FUEL_COST_ID']], self.df_ntndp_gas_cost, how='left',
                               left_on=['FUEL_COST_ID'], right_index=True)

        # Rename columns
        new_columns = {i: int(i.split('-')[0]) for i in df_existing.columns if '-' in i}
        df_existing = df_existing.rename(columns=new_columns)

        # Generators missing cost information for all years
        missing_duids = df_existing[df_existing.drop('FUEL_COST_ID', axis=1).isna().all(axis=1)].index

        # Join fuel cost IDs for missing generators
        df_missing = df_existing.reindex(missing_duids)[['FUEL_COST_ID']]

        # Copy ACIL Allen fuel cost profiles with reset index
        df_acil_cost = self.df_acil_fuel_cost.reset_index().copy()

        # Only consider fuel cost information for liquid fuel
        mask_fuel_type = df_acil_cost['FUEL_COST_ID'] == 'Liquid Fuel'

        # Screen scenarios
        if self.scenario == 'neutral':
            mask_scenario = df_acil_cost['Scenario'] == 'Medium'

        elif self.scenario == 'low':
            mask_scenario = df_acil_cost['Scenario'] == 'Low'

        else:
            raise(Exception(f"Unexpected scenario encountered {scenario}."))

        # Only retain row corresponding to liquid fuel type and given scenario
        df_acil_liquid_cost_profile = df_acil_cost.loc[mask_fuel_type & mask_scenario, :].set_index('FUEL_COST_ID').copy()

        # Merge cost information for liquid fuel units
        df_liquid_cost_profile = pd.merge(df_missing, df_acil_liquid_cost_profile, how='left', left_on='FUEL_COST_ID', right_index=True).drop(['FUEL_COST_ID', 'Scenario'], axis=1)

        # Update cost profile information for liquid fuel generators that have
        # values missing in the NTNDP database
        df_existing.update(df_liquid_cost_profile)

        # Drop fuel cost ID column
        df_existing = df_existing.drop('FUEL_COST_ID', axis=1)

        # Check no rows missing all values for all years
        assert not df_existing.isna().all(axis=1).any(), 'At least one generator with no fuel costs for entire horizon'

        # Locations where zeros are identified in matrix
        # Note: some gas generators have their fuel cost = 0 (problem with NTNDP spreadsheet)
        # must identify these entries accordingly.
        zeros = np.where(df_existing == 0)

        # Denote these as missing values
        df_existing.values[zeros[0], zeros[1]] = np.nan

        # Forward fill missing values, then backfill
        df_existing = df_existing.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

        # Check no missing values
        assert not df_existing.isna().any().any(), 'Missing values for gas and liquid fuel cost profile'

        return df_existing
    
    def get_existing_unit_fom_costs(self):
        """Get FOM costs for existing generators"""
        
        # Join FOM cost IDs to dataframe summarising unit information
        df_tmp = self.df_g.join(self.df_fom_cost_map[['FOM_COST_ID']], how='left').copy()
        
        # Use foreign key to join NTNDP FOM costs
        df_o = pd.merge(df_tmp, self.df_ntndp_fom, how='left', left_on='FOM_COST_ID', right_index=True)[['FOM']]

        # Check that no values are missing
        assert not df_o['FOM'].isna().any(), 'Missing FOM cost values'
        
        return df_o
    
    def get_all_fom_costs(self):
        """Get FOM costs for all unit and place in a single DataFrame"""

        # FOM costs for different types of units
        fom = [self.get_candidate_coal_fom_cost(),
               self.get_candidate_gas_fom_cost(),
               self.get_candidate_solar_fom_cost(),
               self.get_candidate_wind_fom_cost(),
               self.get_existing_unit_fom_costs()]

        # Concatenate FOM costs
        df_fom = pd.concat(fom)

        # Check no missing values
        assert not df_fom.isna().any().any(), 'Missing FOM cost values'
        
        return df_fom
    
    def get_all_vom_costs(self):
        """Get VOM costs for all unit and place in a single DataFrame"""

        # VOM costs for different types of units
        vom = [self.get_candidate_coal_vom_cost(),
               self.get_candidate_gas_vom_cost(),
               self.get_candidate_solar_vom_cost(),
               self.get_candidate_wind_vom_cost(),
               self.df_g[['VOM']]]

        # Concatenate VOM costs
        df_vom = pd.concat(vom)

        # Check no missing values
        assert not df_vom.isna().any().any(), 'Missing VOM cost values'
        
        return df_vom
    
    def get_all_heat_rates(self):
        """Get heat rates for all units and place in a single DataFrame"""

        # Heat rates for different types of units
        heat_rates = [self.get_candidate_coal_heat_rates(),
                      self.get_candidate_gas_heat_rates(),
                      self.get_candidate_solar_heat_rates(),
                      self.get_candidate_wind_heat_rates(),
                      self.df_g[['HEAT_RATE']]]

        # Concatenate heat rates
        df_heat_rates = pd.concat(heat_rates)

        # Check no missing values
        assert not df_heat_rates.isna().any().any(), 'Missing heat rate values'
        
        return df_heat_rates
    
    def get_all_emissions_rates(self):
        """Get emissions rates for all units and place in a single DataFrame"""
        
        # Emissions rates for different types of units
        emissions_rates = [self.get_candidate_coal_emissions_rates(),
                           self.get_candidate_gas_emissions_rates(),
                           self.get_candidate_solar_emissions_rates(),
                           self.get_candidate_wind_emissions_rates(),
                           self.df_g[['EMISSIONS']]]

        # Concatenate heat rates
        df_emissions_rates = pd.concat(emissions_rates)

        # Check no missing values
        assert not df_emissions_rates.isna().any().any(), 'Missing emissions rate values'
        
        return df_emissions_rates
    
    def get_static_data(self):
        """Compile parameters for all units that do not vary over time (static parameters)"""
        
        # FOM costs
        df_fom = self.get_all_fom_costs()
        
        # VOM costs
        df_vom = self.get_all_vom_costs()
        
        # Heat rates
        df_heat_rates = self.get_all_heat_rates()
        
        # Emissions rates
        df_emissions_rates = self.get_all_emissions_rates()
        
        # Concatenate in single DataFrame
        df_static = pd.concat([df_fom, df_vom, df_heat_rates, df_emissions_rates], axis=1)
        
        # Check no missing values
        assert not df_static.isna().any().any(), 'Missing static values'
        
        return df_static
    
    def get_time_varying_fuel_costs(self):
        """Compile fuel cost data for all units (costs vary over time)"""
        
        # All fuel cost information in a single DataFrame
        df_fuel_costs = pd.concat([self.get_candidate_coal_fuel_cost_profiles(),
                                   self.get_candidate_gas_fuel_cost_profiles(),
                                   self.get_existing_coal_fuel_cost_profiles(),
                                   self.get_existing_gas_and_liquid_fuel_cost_profiles()],
                                  sort=False)

        # Reindex so all units included. Fill forward along rows (2041 fuel cost value missing for some units).
        # Then fill missing entries with 0. Assume fuel cost for wind and solar generators = 0 for entire horizon.
        # Will check later that no thermal plant have fuel costs=0.
        df_fuel_costs = df_fuel_costs.reindex(self.get_all_units()).fillna(method='ffill', axis=1).fillna(0)
        
        return df_fuel_costs
    
    def get_time_varying_build_costs(self):
        """Compile build cost data into a single DataFrame"""
        
        # All candidate units which can be invested in
        df_candidate_units = pd.concat([self.get_candidate_coal_units()[['NTNDP_UNIT_ID']],
                                        self.get_candidate_gas_units()[['NTNDP_UNIT_ID']],
                                        self.get_candidate_wind_units()[['NTNDP_UNIT_ID']],
                                        self.get_candidate_solar_units()[['NTNDP_UNIT_ID']]])

        # candidate_units
        df_build_cost = pd.merge(df_candidate_units, self.df_ntndp_build_cost, how='left', 
                                 left_on='NTNDP_UNIT_ID', right_index=True).drop('NTNDP_UNIT_ID', axis=1)

        # Update column names. Assume 2016-17 applies from Jan 1 2016 - Dec 31 2016
        new_cols = {i: int(i.split('-')[0]) for i in df_build_cost.columns}
        df_build_cost = df_build_cost.rename(columns=new_cols)

        # Check for missing values
        assert not df_build_cost.isna().any().any(), 'Missing build cost values'

        return df_build_cost
    

# Paths
# -----
# Directory containing core data files
data_dir = os.path.join(os.path.curdir, os.path.pardir, os.path.pardir, 'data')

Dataset = ConstructDataset(data_dir)
self = Dataset

In [3]:
self.get_static_data()

,FOM,VOM,HEAT_RATE,EMISSIONS
NNS-COAL-SC,50.500000,4.00000,8.670520,0.78902
NNS-COAL-SC-CCS,73.200000,9.00000,11.523688,1.04866
CQ-COAL-SC,50.500000,4.00000,8.670520,0.80636
CQ-COAL-SC-CCS,73.200000,9.00000,11.523688,1.07170
NQ-COAL-SC,50.500000,4.00000,8.670520,0.80636
NQ-COAL-SC-CCS,73.200000,9.00000,11.523688,1.07170
SEQ-COAL-SC,50.500000,4.00000,8.670520,0.80636
SEQ-COAL-SC-CCS,73.200000,9.00000,11.523688,1.07170
SWQ-COAL-SC,50.500000,4.00000,8.670520,0.80636
SWQ-COAL-SC-CCS,73.200000,9.00000,11.523688,1.07170


In [4]:
self.get_time_varying_fuel_costs()

,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,...,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041
ADE-OCGT,8.582271,9.029998,9.658416,9.660450,9.661985,9.662545,9.662722,10.600222,10.600222,10.600222,...,10.600222,10.600222,10.600222,10.600222,10.600222,10.600222,10.600222,10.600222,10.600222,10.600222
ADE-WIND,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AGLHAL,8.405480,8.853207,9.543345,9.549197,9.542662,9.548231,9.608403,10.508422,10.505110,10.503210,...,10.500640,10.500915,10.500915,10.501313,10.501313,10.501313,10.501313,10.501313,10.501313,10.501313
AGLSOM,7.730493,8.377615,9.956760,9.956946,9.957215,9.956736,9.958334,10.895901,10.895982,10.895923,...,11.095786,11.095735,11.095760,11.095769,11.095769,11.095769,11.095769,11.095769,11.095769,11.095769
ANGAST1,35.870805,34.774634,35.104366,35.318213,35.014467,34.564389,34.030942,33.495469,33.005290,32.552812,...,30.221078,29.976614,29.748754,29.532615,29.324503,29.122983,28.926954,28.734433,28.545267,28.183519
ARWF1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BALDHWF1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BARCALDN,6.600000,6.600000,6.600000,6.600000,6.600000,6.600000,7.600000,7.600000,7.600000,7.600000,...,7.600000,7.600000,7.600000,7.600000,7.600000,7.600000,7.600000,7.600000,7.600000,7.600000
BARRON-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BARRON-2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
self.get_time_varying_build_costs()

,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
UNIT_ID,,,,,,,,,,,,,,,,,,,,,
NNS-COAL-SC,3216.290666,3200.959213,3185.704417,3170.525895,3155.423265,3140.396149,3125.444168,3110.566947,3095.764113,3081.035292,...,2855.143696,2841.617978,2828.159888,2814.769089,2801.445243,2788.188017,2774.997077,2761.872092,2748.812731,2735.818667
NNS-COAL-SC-CCS,7054.058285,7031.491280,7009.037110,6986.695211,6964.465021,6942.345983,6694.827028,6456.327878,6226.519843,6005.086212,...,4878.799772,4855.446843,4829.821832,4804.966399,4779.078712,4753.660237,4730.684968,4708.476983,4686.237117,4665.149174
CQ-COAL-SC,3146.290666,3130.959213,3115.704417,3100.525895,3085.423265,3070.396149,3055.444168,3040.566947,3025.764113,3011.035292,...,2785.143696,2771.617978,2758.159888,2744.769089,2731.445243,2718.188017,2704.997077,2691.872092,2678.812731,2665.818667
CQ-COAL-SC-CCS,6984.058285,6961.491280,6939.037110,6916.695211,6894.465021,6872.345983,6624.827028,6386.327878,6156.519843,5935.086212,...,4808.799772,4785.446843,4759.821832,4734.966399,4709.078712,4683.660237,4660.684968,4638.476983,4616.237117,4595.149174
NQ-COAL-SC,3356.290666,3340.959213,3325.704417,3310.525895,3295.423265,3280.396149,3265.444168,3250.566947,3235.764113,3221.035292,...,2995.143696,2981.617978,2968.159888,2954.769089,2941.445243,2928.188017,2914.997077,2901.872092,2888.812731,2875.818667
NQ-COAL-SC-CCS,7194.058285,7171.491280,7149.037110,7126.695211,7104.465021,7082.345983,6834.827028,6596.327878,6366.519843,6145.086212,...,5018.799772,4995.446843,4969.821832,4944.966399,4919.078712,4893.660237,4870.684968,4848.476983,4826.237117,4805.149174
SEQ-COAL-SC,3066.290666,3050.959213,3035.704417,3020.525895,3005.423265,2990.396149,2975.444168,2960.566947,2945.764113,2931.035292,...,2705.143696,2691.617978,2678.159888,2664.769089,2651.445243,2638.188017,2624.997077,2611.872092,2598.812731,2585.818667
SEQ-COAL-SC-CCS,6904.058285,6881.491280,6859.037110,6836.695211,6814.465021,6792.345983,6544.827028,6306.327878,6076.519843,5855.086212,...,4728.799772,4705.446843,4679.821832,4654.966399,4629.078712,4603.660237,4580.684968,4558.476983,4536.237117,4515.149174
SWQ-COAL-SC,3186.290666,3170.959213,3155.704417,3140.525895,3125.423265,3110.396149,3095.444168,3080.566947,3065.764113,3051.035292,...,2825.143696,2811.617978,2798.159888,2784.769089,2771.445243,2758.188017,2744.997077,2731.872092,2718.812731,2705.818667
